# MNIST Regression

target label이 one hot vector가 아닌 1개의 숫자 [0, 9]

## Setup
### The MNIST Data

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
print mnist.train.labels[0]

7


In [3]:
import tensorflow as tf
sess = tf.InteractiveSession()

## Inputs and Placeholders

In [4]:
images = tf.placeholder('float', shape=[None, 784])
labels = tf.placeholder('float', shape=[None, 1])

## Build a Multilayer Convolution Layers

In [5]:
keep_prob = tf.placeholder('float')
def inference(x):
    with tf.name_scope('conv1'):
        weights = tf.Variable( tf.truncated_normal([5, 5, 1, 32], stddev=0.1) )
        bias = tf.Variable( tf.constant(0.1, shape=[32]) )
    
        reshaped_input = tf.reshape(x, [-1, 28,28,1])
        _ = tf.nn.conv2d(reshaped_input, weights, strides=[1, 1, 1, 1], padding='SAME') + bias
        _ = tf.nn.relu( _ )
        _ = tf.nn.max_pool(_, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        conv1_out = _

    with tf.name_scope('conv2'):
        weights = tf.Variable( tf.truncated_normal([5, 5, 32, 64], stddev=0.1) )
        bias = tf.Variable( tf.constant(0.1, shape=[64]) )
    
        _ = tf.nn.conv2d(conv1_out, weights, strides=[1, 1, 1, 1], padding='SAME') + bias
        _ = tf.nn.relu( _ )
        _ = tf.nn.max_pool(_, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')    
        conv2_out = _

    with tf.name_scope('fullconn'):
        weights = tf.Variable( tf.truncated_normal([7*7*64, 1024], stddev=0.1) )
        bias = tf.Variable( tf.constant(0.1, shape=[1024]) )
    
        reshaped_input = tf.reshape(conv2_out, [-1, 7*7*64])
        _ = tf.matmul(reshaped_input, weights) + bias
        _ = tf.nn.relu( _ )
        fullconn_out = _
    
        # dropout
        dropout_out = tf.nn.dropout(fullconn_out, keep_prob)

    with tf.name_scope('output'):
        weights = tf.Variable(  tf.truncated_normal([1024, 1], stddev=0.1) )
        bias = tf.Variable( tf.constant(0.1, shape=[1]) )
    
        _ = tf.matmul(dropout_out, weights) + bias
        y = _
        
    return y

In [6]:
cross_entropy = ( 0.5 * tf.pow((inference(images) - labels), 2) )
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [7]:
sess.run(tf.initialize_all_variables())

In [8]:
import numpy as np
for i in range(1000):
    batch = mnist.train.next_batch(50)
    _labels = np.reshape(batch[1], [50, 1]).astype('float')
    if i%100 == 0:
        print ("step %d, training loss %g"%(i, np.mean(cross_entropy.eval(feed_dict={
            images: batch[0], labels: _labels, keep_prob: 1.0}))))
    train_step.run(feed_dict={images: batch[0], labels: _labels, keep_prob: 0.5})
    
print("test loss %g"% np.mean( cross_entropy.eval(feed_dict={
    images: mnist.test.images, labels: np.reshape(mnist.test.labels, [-1, 1]).astype('float'), keep_prob: 1.0}) ))


step 0, training loss 13.6601
step 100, training loss 2.17481
step 200, training loss 0.670446
step 300, training loss 1.57771
step 400, training loss 0.879913
step 500, training loss 0.711898
step 600, training loss 0.91927
step 700, training loss 0.636307
step 800, training loss 0.687924
step 900, training loss 0.606165
test loss 0.667562
